*flask_login 동작 방식*
1. 사용자 로그인
1. 사용자 로그인 정보를 User class에서 가져와 LoginManager() 에 추가하여 세션 생성
1. current_user 객체에 해당 객체가 생성
1. 로그인 후 웹페이지로 접근 시, 전달받은 세션 정보를 기반으로 접근
    - @login_required 데코레이터는 세션이 없으면 접근 불가능
1. 사용자 로그아웃시 LoginManager()에서 해당 id 제거

In [ ]:
from flask import Flask, render_template, redirect, url_for
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin, current_user
from requests import request

app = Flask(__name__)
app.secret_key = 'your_secret_key'

login_manager = LoginManager()
login_manager.init_app(app)

users = {
    'user1': {'password': 'password1'},
    'user2': {'password': 'password2'}
}

class User(UserMixin):
    pass

@login_manager.user_loader
def user_loader(username):
    if username not in users:
        return

    user = User()
    user.id = username
    return user

@app.route('/')
def index():
    if current_user.is_authenticated:
        return f'Logged in as {current_user.id}. <a href="/logout">Logout</a>'
    return 'You are not logged in. <a href="/login">Login</a>'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return redirect(url_for('index'))

    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        if username in users and password == users[username]['password']:
            user = User()
            user.id = username
            login_user(user)
            return redirect(url_for('index'))

        return render_template('login.html', error='Invalid username or password')

    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('index'))
